In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


def perform_eda(file_path, selected_cryptos):
    """
    Perform EDA on selected cryptocurrencies and return a dictionary of interactive plots.
    """
    # Load the dataset
    crypto_data = pd.read_csv(file_path)
    crypto_data['Date'] = pd.to_datetime(crypto_data['Date'])

    # Filter for selected cryptocurrencies
    filtered_data = crypto_data[crypto_data['Symbol'].isin(selected_cryptos)]

    # Initialize a dictionary to store plots
    eda_plots = {}

    # 1. Temporal Structure Analysis 
    temporal_fig = go.Figure()
    for crypto in selected_cryptos:
        crypto_series = filtered_data[filtered_data['Symbol'] == crypto]
        temporal_fig.add_trace(go.Scatter(
            x=crypto_series['Date'],
            y=crypto_series['Adj'],
            mode='lines',
            name=f"{crypto} Prices"
        ))
    temporal_fig.update_layout(
        title="Temporal Structure: Adjusted Closing Prices",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white"
    )
    eda_plots['temporal_structure'] = temporal_fig

    # 2. Distribution Visualization (KDE)
    distribution_fig = go.Figure()
    for crypto in selected_cryptos:
        crypto_series = filtered_data[filtered_data['Symbol'] == crypto]['Adj']
        fig = px.histogram(crypto_series, nbins=50, opacity=0.7, histnorm='density', title=f"{crypto} KDE Plot")
        fig.update_traces(name=f"{crypto}", showlegend=True)
        distribution_fig.add_traces(fig.data)
    distribution_fig.update_layout(
        title="Distribution of Adjusted Closing Prices (KDE)",
        xaxis_title="Price (USD)",
        yaxis_title="Density",
        template="plotly_white"
    )
    eda_plots['distribution'] = distribution_fig

    # 3. Rolling Volatility Analysis
    volatility_fig = go.Figure()
    for crypto in selected_cryptos:
        crypto_series = filtered_data[filtered_data['Symbol'] == crypto]
        volatility = crypto_series['Adj'].rolling(window=7).std()
        volatility_fig.add_trace(go.Scatter(
            x=crypto_series['Date'],
            y=volatility,
            mode='lines',
            name=f"{crypto} Volatility (7-Day)"
        ))
    volatility_fig.update_layout(
        title="7-Day Rolling Volatility Analysis",
        xaxis_title="Date",
        yaxis_title="Volatility",
        template="plotly_white"
    )
    eda_plots['volatility'] = volatility_fig

    # 4. Candlestick Chart with Moving Averages
    for crypto in selected_cryptos:
        crypto_series = filtered_data[filtered_data['Symbol'] == crypto]
        fig = go.Figure()

        # Add candlestick
        fig.add_trace(go.Candlestick(
            x=crypto_series['Date'],
            open=crypto_series['Open'],
            high=crypto_series['High'],
            low=crypto_series['Low'],
            close=crypto_series['Adj'],
            name=f"{crypto} OHLC"
        ))

        # Add moving averages for trend analysis
        crypto_series['Moving_Avg_7'] = crypto_series['Adj'].rolling(window=7).mean()
        crypto_series['Moving_Avg_14'] = crypto_series['Adj'].rolling(window=14).mean()

        fig.add_trace(go.Scatter(
            x=crypto_series['Date'],
            y=crypto_series['Moving_Avg_7'],
            mode='lines',
            name='7-Day MA',
            line=dict(color='blue')
        ))

        fig.add_trace(go.Scatter(
            x=crypto_series['Date'],
            y=crypto_series['Moving_Avg_14'],
            mode='lines',
            name='14-Day MA',
            line=dict(color='purple')
        ))

        fig.update_layout(
            title=f"Candlestick Chart with Moving Averages for {crypto}",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            xaxis_rangeslider_visible=True,
            template="plotly_white"
        )

        eda_plots[f'candlestick_chart_{crypto}'] = fig

    # 5. Seasonal Decomposition Plot 
    from statsmodels.tsa.seasonal import seasonal_decompose
    decomposition_figures = {}
    for crypto in selected_cryptos:
        crypto_series = filtered_data[filtered_data['Symbol'] == crypto].set_index('Date')['Adj']
        decomposed = seasonal_decompose(crypto_series, model='additive', period=30)

        fig = go.Figure()

        # Observed
        fig.add_trace(go.Scatter(x=crypto_series.index, y=decomposed.observed, mode='lines', name='Observed'))

        # Trend
        fig.add_trace(go.Scatter(x=crypto_series.index, y=decomposed.trend, mode='lines', name='Trend'))

        # Seasonal
        fig.add_trace(go.Scatter(x=crypto_series.index, y=decomposed.seasonal, mode='lines', name='Seasonal'))

        # Residual
        fig.add_trace(go.Scatter(x=crypto_series.index, y=decomposed.resid, mode='lines', name='Residual'))

        fig.update_layout(
            title=f"Seasonal Decomposition of {crypto}",
            xaxis_title="Date",
            yaxis_title="Value",
            template="plotly_white"
        )

        decomposition_figures[f'seasonal_decomposition_{crypto}'] = fig

    eda_plots.update(decomposition_figures)

    # 6. Daily Returns Analysis ( Histogram)
    daily_returns_fig = go.Figure()
    filtered_data['Daily_Return'] = filtered_data.groupby('Symbol')['Adj'].pct_change()
    for crypto in selected_cryptos:
        daily_returns = filtered_data[filtered_data['Symbol'] == crypto]['Daily_Return']
        fig = px.histogram(daily_returns, nbins=50, opacity=0.7, histnorm='density', title=f"{crypto} Daily Returns")
        fig.update_traces(name=f"{crypto}", showlegend=True)
        daily_returns_fig.add_traces(fig.data)
    daily_returns_fig.update_layout(
        title="Daily Returns Distribution",
        xaxis_title="Daily Return",
        yaxis_title="Density",
        template="plotly_white"
    )
    eda_plots['daily_returns'] = daily_returns_fig

    return eda_plots